# Quick Test Training - 快速测试训练

使用3张nasi_lemak图像测试训练流程

**注意**: 这只是测试流程，不会得到好的模型！

---


## 1. 环境检查


In [ ]:
# Install dependencies
%pip install ultralytics pillow pyyaml -q

import torch
import ultralytics
from pathlib import Path

print(f"✅ Ultralytics: {ultralytics.__version__}")
print(f"✅ PyTorch: {torch.__version__}")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Device: {device}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
else:
    print("   ⚠️  使用CPU训练会很慢，但可以验证流程")


## 2. 准备测试数据集


In [ ]:
import os
import shutil
from pathlib import Path

# 创建测试数据集目录
dataset_root = Path('test_dataset')
dataset_root.mkdir(exist_ok=True)

# 创建train/val/test分割
for split in ['train', 'valid', 'test']:
    (dataset_root / split / 'images').mkdir(parents=True, exist_ok=True)
    (dataset_root / split / 'labels').mkdir(parents=True, exist_ok=True)

# 从test_images复制图像
test_images_dir = Path('../../test_images/nasi_lemak')
images = list(test_images_dir.glob('*.jpg'))

print(f"找到 {len(images)} 张测试图像")

# 分配：2张训练，1张验证
if len(images) >= 3:
    train_images = images[:2]
    val_images = images[2:3]
    
    for img in train_images:
        shutil.copy(img, dataset_root / 'train' / 'images' / img.name)
    
    for img in val_images:
        shutil.copy(img, dataset_root / 'valid' / 'images' / img.name)
        shutil.copy(img, dataset_root / 'test' / 'images' / img.name)
    
    print(f"✅ 训练集: {len(train_images)} 张")
    print(f"✅ 验证集: {len(val_images)} 张")
else:
    print("❌ 图像不足！")


## 3. 创建虚拟标注

为每张图像创建简单标注（覆盖整张图）


In [ ]:
from PIL import Image

def create_dummy_label(image_path, label_path, class_id=0):
    """创建覆盖整张图的虚拟标注"""
    img = Image.open(image_path)
    width, height = img.size
    
    # YOLO格式: class_id x_center y_center width height
    label_content = f"{class_id} 0.5 0.5 0.9 0.9\n"
    
    with open(label_path, 'w') as f:
        f.write(label_content)

# 为所有图像创建标注
for split in ['train', 'valid', 'test']:
    images_dir = dataset_root / split / 'images'
    labels_dir = dataset_root / split / 'labels'
    
    for img_path in images_dir.glob('*.jpg'):
        label_path = labels_dir / f"{img_path.stem}.txt"
        create_dummy_label(img_path, label_path, class_id=0)
    
    label_count = len(list(labels_dir.glob('*.txt')))
    print(f"✅ {split}: 创建了 {label_count} 个标注")

print("\n⚠️  注意: 这些是虚拟标注，仅用于测试！")


## 4. 创建data.yaml配置


In [ ]:
import yaml

# YOLO配置文件
data_config = {
    'path': str(dataset_root.absolute()),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['nasi_lemak']
}

data_yaml = dataset_root / 'data.yaml'
with open(data_yaml, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✅ data.yaml 已创建")
print(f"\n配置:")
print(yaml.dump(data_config, default_flow_style=False))


## 5. 快速训练测试

⏱️ 预计时间: CPU 5-10分钟, GPU 1-2分钟


In [ ]:
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

print("🚀 开始测试训练...")
print()

# 加载最小模型
model = YOLO('yolov8n.pt')

# 快速训练（仅测试）
try:
    results = model.train(
        data=str(data_yaml),
        epochs=3,              # 只训练3轮
        imgsz=640,
        batch=1,
        device=device,
        project='test_runs',
        name='quick_test',
        patience=0,
        save=True,
        plots=False,
        verbose=True,
        # 禁用数据增强
        hsv_h=0,
        hsv_s=0,
        hsv_v=0,
        mosaic=0
    )
    
    print("\n✅ 训练流程测试成功！")
    print("\n⚠️  重要:")
    print("   - 这只是测试，模型不会有实际能力")
    print("   - 需要1000+张图像才能训练出好模型")
    print("   - 继续Week 2数据收集！")
    
except Exception as e:
    print(f"\n❌ 出错: {e}")
    print("\n可能原因:")
    print("   1. 内存不足")
    print("   2. 图像格式问题")
    print("   3. PyTorch安装问题")


## 6. 测试推理


In [ ]:
# 加载训练好的模型
trained_model = YOLO('test_runs/quick_test/weights/best.pt')

# 测试推理
test_img = list((dataset_root / 'test/images').glob('*.jpg'))[0]
results = trained_model(test_img)

print("✅ 推理测试成功！")
print(f"\n模型输出: {results[0].boxes}")
print("\n（检测可能不准确，这是正常的）")

# 显示图像
import matplotlib.pyplot as plt
img = Image.open(test_img)
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')
plt.title('Test Image')
plt.show()


## 总结

### ✅ 测试成功说明
- 训练环境配置正确
- 训练流程可以运行
- 可以继续数据收集

### 📊 对比

| 项目 | 测试 | 真实训练 |
|------|------|---------|
| 图像 | 3张 | 1000+张 |
| 类别 | 1类 | 20类 |
| Epochs | 3 | 100 |
| 时间 | 5-10分钟 | 2-4小时 |
| mAP50 | ~5% | >80% |

### 🚀 下一步
1. 继续Week 2数据收集
2. 收集1000+张图像
3. 上传Roboflow标注
4. Week 3正式训练

---

**现在去收集数据吧！** 🎯
